#### Import Necessary Libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import seaborn as sns

adbCodes=pd.read_csv(os.path.join(os.getcwd(),"data","ADB_CountryCodes.csv"))
NewData=adbCodes[['name','ISO','ADB_DEP','TE_Name','CurrencyCode']].copy()
NewData[['currency_Price','currency_YOYChange','currency_Date']]=""
NewData[['CPI_Inflation_YoY','Inflation_Change','CPI_Inflation_Date']]=""
NewData[['Food_Inflation_YoY','Food_Inflation_Change','Food_Inflation_Date']]=""



#### Download Currency data from Trading Economics

In [71]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}

for i in range(0,NewData.shape[0]):
    URL="https://tradingeconomics.com/"+NewData['TE_Name'].iloc[i]+"/currency"
    response=requests.get(URL,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    tables=soup.find_all('table','table')
    if len(tables)>2:
        df1=pd.read_html(str(tables[1]))[0]
        if 'Crosses' in list(df1.columns):
            NewData['currency_Price'].iloc[i]=float(df1[df1['Crosses']=="USD"+NewData['CurrencyCode'].iloc[i]]['Price'])
            NewData['currency_YOYChange'].iloc[i]=df1[df1['Crosses']=="USD"+NewData['CurrencyCode'].iloc[i]]['Year'].iloc[0]
            NewData['currency_Date'].iloc[i]=df1[df1['Crosses']=="USD"+NewData['CurrencyCode'].iloc[i]]['Date'].iloc[0]
    print(i,end=" ")
            
            
            

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 

#### Download Inflation, CPI Change Data from Trading Economics

In [72]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
for i in range(0,NewData.shape[0]):
    URL="https://tradingeconomics.com/"+NewData['TE_Name'].iloc[i]+"/inflation-cpi"
    response=requests.get(URL,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    tables=soup.find_all('table','table')
    tableno=""
    current=""
    prev=""
    change=""
    if len(tables)>=1:
        for j in range(0,len(tables)):
            df1=pd.read_html(str(tables[j]))[0]
            if 'Related' in list(df1.columns):
                tableno=j
                df1=pd.read_html(str(tables[tableno]))[0]
                current=df1[df1['Related']=="Inflation Rate"]['Last'].iloc[0]
                prev=df1[df1['Related']=="Inflation Rate"]['Previous'].iloc[0]
                change=current-prev 
                NewData['CPI_Inflation_YoY'].iloc[i]=current
                NewData['Inflation_Change'].iloc[i]=change
                NewData['CPI_Inflation_Date'].iloc[i]=df1[df1['Related']=="Inflation Rate"]['Reference'].iloc[0]
                print(i, end=" ")
                
            

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 16 23 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 

#### Download Food Inflation Data from Trading Economics

In [73]:
#Set Necessary headers to read data
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
for i in range(0,NewData.shape[0]):
    URL="https://tradingeconomics.com/"+NewData['TE_Name'].iloc[i]+"/food-inflation"
    response=requests.get(URL,headers=headers)
    soup=BeautifulSoup(response.content,'html.parser')
    tables=soup.find_all('table','table')
    tableno=""
    current=""
    prev=""
    change=""
    if len(tables)>=1:
        for j in range(0,len(tables)):
            df1=pd.read_html(str(tables[j]))[0]
            if 'Related' in list(df1.columns):
                tableno=j
                df1=pd.read_html(str(tables[tableno]))[0]
                current=df1[df1['Related']=="Food Inflation"]['Last'].iloc[0]
                prev=df1[df1['Related']=="Food Inflation"]['Previous'].iloc[0]
                change=current-prev 
                NewData['Food_Inflation_YoY'].iloc[i]=current
                NewData['Food_Inflation_Change'].iloc[i]=change
                NewData['Food_Inflation_Date'].iloc[i]=df1[df1['Related']=="Food Inflation"]['Reference'].iloc[0]
                print(i, end=" ")
                

0 1 2 3 4 5 6 7 9 10 11 12 13 14 16 23 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 45 

#### Download Commodity Prices

In [15]:
url="https://thedocs.worldbank.org/en/doc/5d903e848db1d1b83e0ec8f744e55570-0350012021/related/CMO-Historical-Data-Monthly.xlsx"
r = requests.get(url)
open('temp.xls', 'wb').write(r.content)
df = pd.read_excel('temp.xls',sheet_name="Monthly Prices",skiprows=4, usecols="A:BT")

In [18]:
df.tail(1)

,Unnamed: 0,"Crude oil, average","Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI","Coal, Australian","Coal, South African","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
750,2022M05,110.096667,112.37,108.32,109.6,..,280,8.1373,29.84629,16.5267,...,2830.32,131.21,9377.15,2142.48,35769.39,28062.55,3751.48,1848.5,958.38,21.905


In [23]:
df.columns

Index(['Unnamed: 0', 'Crude oil, average', 'Crude oil, Brent',
       'Crude oil, Dubai', 'Crude oil, WTI', 'Coal, Australian',
       'Coal, South African', 'Natural gas, US', 'Natural gas, Europe',
       'Liquefied natural gas, Japan', 'Natural gas index', 'Cocoa',
       'Coffee, Arabica', 'Coffee, Robusta', 'Tea, avg 3 auctions',
       'Tea, Colombo', 'Tea, Kolkata', 'Tea, Mombasa', 'Coconut oil',
       'Groundnuts', 'Fish meal', 'Groundnut oil', 'Palm oil',
       'Palm kernel oil', 'Soybeans', 'Soybean oil', 'Soybean meal',
       'Rapeseed oil', 'Sunflower oil', 'Barley', 'Maize', 'Sorghum',
       'Rice, Thai 5% ', 'Rice, Thai 25% ', 'Rice, Thai A.1',
       'Rice, Viet Namese 5%', 'Wheat, US SRW', 'Wheat, US HRW',
       'Banana, Europe', 'Banana, US', 'Orange', 'Beef', 'Meat, chicken',
       'Meat, sheep', 'Shrimps, Mexican', 'Sugar, EU', 'Sugar, US',
       'Sugar, world', 'Tobacco, US import u.v.', 'Logs, Cameroon',
       'Logs, Malaysian', 'Sawnwood, Cameroon', 'Sawnw

In [25]:
ColsEnergy=['Crude oil, average','Natural gas, US']
ColsOils=['Palm oil','Soybeans', 'Soybean oil','Sunflower oil']
ColsCereals=['Soybeans','Maize','Rice, Thai 5% ', 'Rice, Thai 25% ', 'Rice, Thai A.1',
       'Rice, Viet Namese 5%', 'Wheat, US SRW', 'Wheat, US HRW']
ColsOthers=['Sugar, world','Cotton, A Index']
ColsFert=['Phosphate rock', 'DAP', 'TSP', 'Urea ', 'Potassium chloride']
